### Model Compression Comparative Study

**Hardware Platforms:**
- Google Colab GPU (Cloud)
- HP Omnibook Intel Arc GPU (Local)
- Lenovo ThinkCentre (Intel Core i5 Desktop)
---


In [5]:
# COLAB ONLY:
# !pip install torch torchvision torchaudio
# !pip install matplotlib pandas numpy

This block below imports PyTorch and its common submodules along with supporting libraries, sets fixed random seeds for reproducibility, and prints the PyTorch version.

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
from datetime import datetime

torch.manual_seed(42)
np.random.seed(42)

print(f"PyTorch version: {torch.__version__}")

PyTorch version: 2.10.0+cpu


This helper function checks for an Intel XPU first, then a CUDA GPU, and falls back to CPU and prints which device it selects. Important since this code is utilized on multiple platforms.

In [ ]:
def get_device():
    """
    Auto-detect the best available device for training.
    Priority: Intel XPU > CUDA > CPU
    """
    try:
        import intel_extension_for_pytorch as ipex
        if torch.xpu.is_available():
            print("Using Intel XPU (Arc GPU)")
            return torch.device('xpu')
    except:
        pass
    
    if torch.cuda.is_available():
        print(f"Using CUDA GPU: {torch.cuda.get_device_name(0)}")
        return torch.device('cuda')
    
    print("Using CPU (no GPU detected)")
    return torch.device('cpu')

device = get_device()
print(f"Device: {device}")

Using CPU (no GPU detected)
Device: cpu


This section sets batch parameters, defines normalization transforms for MNIST and CIFAR‑10, downloads both datasets, wraps them in `DataLoader`'s, and prints the counts.

In [ ]:
BATCH_SIZE = 128 
NUM_WORKERS = 2 

mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  
])

cifar_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)) 
])

print("Loading MNIST dataset...")
mnist_train = torchvision.datasets.MNIST(
    root='./data', 
    train=True, 
    download=True, 
    transform=mnist_transform
)
mnist_test = torchvision.datasets.MNIST(
    root='./data', 
    train=False, 
    download=True, 
    transform=mnist_transform
)

print("Loading CIFAR-10 dataset...")
cifar_train = torchvision.datasets.CIFAR10(
    root='./data', 
    train=True, 
    download=True, 
    transform=cifar_transform
)
cifar_test = torchvision.datasets.CIFAR10(
    root='./data', 
    train=False, 
    download=True, 
    transform=cifar_transform
)

mnist_train_loader = DataLoader(mnist_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
mnist_test_loader = DataLoader(mnist_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

cifar_train_loader = DataLoader(cifar_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
cifar_test_loader = DataLoader(cifar_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

print(f"\n✓ MNIST: {len(mnist_train)} train, {len(mnist_test)} test")
print(f"✓ CIFAR-10: {len(cifar_train)} train, {len(cifar_test)} test")

Loading MNIST dataset...
Loading CIFAR-10 dataset...


c:\Users\harri\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\torchvision\datasets\cifar.py:83: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  entry = pickle.load(f, encoding="latin1")



✓ MNIST: 60000 train, 10000 test
✓ CIFAR-10: 50000 train, 10000 test


Code below is where 2 simple convolutional neural networks are defined: `SimpleMNIST` for grayscale 28×28 digits and `SimpleCIFAR` for color 32×32 images, both with a few conv/pool layers followed by fully‑connected layers and dropout, and then instances are created and moved to the chosen device.

In [9]:
class SimpleMNIST(nn.Module):
    """
    Simple CNN for MNIST (grayscale 28x28 images)
    
    Architecture:
    - Conv1: 1 -> 32 channels, 3x3 kernel
    - Conv2: 32 -> 64 channels, 3x3 kernel
    - FC1: 9216 -> 128 neurons
    - FC2: 128 -> 10 classes (digits 0-9)
    """
    def __init__(self):
        super(SimpleMNIST, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) 
        x = self.pool(F.relu(self.conv2(x)))  
        x = x.view(-1, 64 * 7 * 7)          
        x = F.relu(self.fc1(x))              
        x = self.dropout(x)                  
        x = self.fc2(x)                     
        return x


class SimpleCIFAR(nn.Module):
    """
    Simple CNN for CIFAR-10 (color 32x32 images)
    
    Architecture:
    - Conv1: 3 -> 64 channels, 3x3 kernel
    - Conv2: 64 -> 128 channels, 3x3 kernel
    - Conv3: 128 -> 256 channels, 3x3 kernel
    - FC1: 4096 -> 512 neurons
    - FC2: 512 -> 10 classes
    """
    def __init__(self):
        super(SimpleCIFAR, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)
        
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  
        x = self.pool(F.relu(self.conv2(x)))  
        x = self.pool(F.relu(self.conv3(x)))  
        x = x.view(-1, 256 * 4 * 4)           
        x = F.relu(self.fc1(x))             
        x = self.dropout(x)                  
        x = self.fc2(x)                    
        return x

print("Creating models.")
mnist_model = SimpleMNIST().to(device)
cifar_model = SimpleCIFAR().to(device)

print(f"MNIST model created with {sum(p.numel() for p in mnist_model.parameters()):,} parameters")
print(f"CIFAR model created with {sum(p.numel() for p in cifar_model.parameters()):,} parameters")

Creating models.
MNIST model created with 421,642 parameters
CIFAR model created with 2,473,610 parameters


The `train_model` function loops over epochs and batches, computes loss/accuracy, performs backpropagation and optimizer steps, and returns the epoch-wise loss history.

In [10]:
def train_model(model, train_loader, criterion, optimizer, epochs=5):
    """
    Train a neural network model.
    
    Args:
        model: The neural network to train
        train_loader: DataLoader with training data
        criterion: Loss function (how wrong predictions are)
        optimizer: Algorithm to update weights
        epochs: Number of times to see the entire dataset
    
    Returns:
        List of losses per epoch
    """
    model.train() 
    losses = []
    
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            
            loss = criterion(output, target)
            
            loss.backward()
            
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()
            
            if batch_idx % 100 == 0:
                print(f'Epoch {epoch+1}/{epochs}, Batch {batch_idx}/{len(train_loader)}, '
                      f'Loss: {loss.item():.4f}, Acc: {100.*correct/total:.2f}%')
        
        epoch_loss = running_loss / len(train_loader)
        losses.append(epoch_loss)
        print(f'\n>>> Epoch {epoch+1} complete. Avg Loss: {epoch_loss:.4f}, '
              f'Train Accuracy: {100.*correct/total:.2f}%\n')
    
    return losses


def evaluate_model(model, test_loader):
    """
    Evaluate model on test data.
    
    Returns:
        accuracy: Percentage of correct predictions (0-100)
    """
    model.eval()  
    correct = 0
    total = 0
    
    with torch.no_grad(): 
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()
    
    accuracy = 100. * correct / total
    return accuracy


def get_model_size(model):
    """
    Calculate model size in megabytes (MB).
    
    Each parameter is a 32-bit float = 4 bytes
    """
    param_size = sum(p.numel() for p in model.parameters()) * 4  
    buffer_size = sum(b.numel() for b in model.buffers()) * 4
    size_mb = (param_size + buffer_size) / 1024**2
    return size_mb


def measure_inference_time(model, test_loader, num_iterations=100):
    """
    Measure average inference time per image (in milliseconds).
    
    Args:
        model: The neural network
        test_loader: DataLoader with test data
        num_iterations: Number of batches to average over
    
    Returns:
        avg_time_ms: Average time per single image in milliseconds
    """
    model.eval()
    times = []
    
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            if i >= num_iterations:
                break
            
            data = data.to(device)
            
            if i == 0:
                _ = model(data)
                continue
            
            start = time.time()
            _ = model(data)
            
            if device.type == 'cuda':
                torch.cuda.synchronize()
            elif device.type == 'xpu':
                torch.xpu.synchronize()
            
            end = time.time()
            
            batch_time = (end - start) * 1000 
            per_image_time = batch_time / data.size(0)
            times.append(per_image_time)
    
    avg_time_ms = np.mean(times)
    return avg_time_ms


def collect_metrics(model, test_loader, model_name):
    """
    Collect all metrics for a model.
    
    Returns:
        Dictionary with accuracy, size, and latency
    """
    print(f"Evaluating {model_name}...")
    
    accuracy = evaluate_model(model, test_loader)
    size_mb = get_model_size(model)
    latency_ms = measure_inference_time(model, test_loader)
    
    metrics = {
        'model_name': model_name,
        'accuracy': accuracy,
        'size_mb': size_mb,
        'latency_ms': latency_ms,
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    print(f"  Accuracy: {accuracy:.2f}%")
    print(f"  Size: {size_mb:.2f} MB")
    print(f"  Latency: {latency_ms:.4f} ms/image")
    print()
    
    return metrics

print("Training and evaluation functions defined")

Training and evaluation functions defined


Here, the script sets the learning rate and number of epochs, initializes a results list, and prints a header announcing the start of baseline model training.

In [ ]:
LEARNING_RATE = 0.001
EPOCHS = 10  

all_results = []

print("TRAINING BASELINE MODELS")
print("-" * 60)

print("\n[1/2] Training MNIST Baseline...\n")
mnist_model = SimpleMNIST().to(device)
mnist_criterion = nn.CrossEntropyLoss()
mnist_optimizer = optim.Adam(mnist_model.parameters(), lr=LEARNING_RATE)

mnist_losses = train_model(mnist_model, mnist_train_loader, mnist_criterion, mnist_optimizer, epochs=EPOCHS)

mnist_metrics = collect_metrics(mnist_model, mnist_test_loader, "MNIST_Baseline")
all_results.append(mnist_metrics)

torch.save(mnist_model.state_dict(), 'mnist_baseline.pth')
print("MNIST model saved to mnist_baseline.pth\n")

print("\n[2/2] Training CIFAR-10 Baseline...\n")
cifar_model = SimpleCIFAR().to(device)
cifar_criterion = nn.CrossEntropyLoss()
cifar_optimizer = optim.Adam(cifar_model.parameters(), lr=LEARNING_RATE)

cifar_losses = train_model(cifar_model, cifar_train_loader, cifar_criterion, cifar_optimizer, epochs=EPOCHS)

cifar_metrics = collect_metrics(cifar_model, cifar_test_loader, "CIFAR_Baseline")
all_results.append(cifar_metrics)

torch.save(cifar_model.state_dict(), 'cifar_baseline.pth')
print("CIFAR model saved to cifar_baseline.pth\n")

print("BASELINE TRAINING COMPLETE!")
print("-" * 60)

# Display results
results_df = pd.DataFrame(all_results)
print("\nBaseline Results:")
print(results_df[['model_name', 'accuracy', 'size_mb', 'latency_ms']])

# Save to CSV
results_df.to_csv('baseline_results.csv', index=False)
print("\nResults saved to baseline_results.csv")

TRAINING BASELINE MODELS
------------------------------------------------------------

[1/2] Training MNIST Baseline...

Epoch 1/10, Batch 0/469, Loss: 2.3213, Acc: 7.03%
Epoch 1/10, Batch 100/469, Loss: 0.3103, Acc: 79.46%
Epoch 1/10, Batch 200/469, Loss: 0.1830, Acc: 86.66%
Epoch 1/10, Batch 300/469, Loss: 0.1263, Acc: 89.71%
Epoch 1/10, Batch 400/469, Loss: 0.2008, Acc: 91.40%

>>> Epoch 1 complete. Avg Loss: 0.2543, Train Accuracy: 92.16%

Epoch 2/10, Batch 0/469, Loss: 0.1128, Acc: 96.88%
Epoch 2/10, Batch 100/469, Loss: 0.0707, Acc: 97.17%
Epoch 2/10, Batch 200/469, Loss: 0.0616, Acc: 97.31%
Epoch 2/10, Batch 300/469, Loss: 0.0709, Acc: 97.31%
Epoch 2/10, Batch 400/469, Loss: 0.0472, Acc: 97.47%

>>> Epoch 2 complete. Avg Loss: 0.0870, Train Accuracy: 97.47%

Epoch 3/10, Batch 0/469, Loss: 0.0381, Acc: 99.22%
Epoch 3/10, Batch 100/469, Loss: 0.0819, Acc: 98.14%
Epoch 3/10, Batch 200/469, Loss: 0.0586, Acc: 97.91%
Epoch 3/10, Batch 300/469, Loss: 0.0723, Acc: 98.05%
Epoch 3/10, Ba